In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',index_col=0,error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

In [2]:
cant_buckets = 15

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

#tenia un nan nose porque
properati.dropna(inplace=True)

In [3]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


### Busco hiper-parametros con grid search

In [5]:
%%notify

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

etc = ExtraTreesClassifier(n_jobs=-1)

bootstrap=[True,False]
max_features =["auto","sqrt","log2",None]

param_grid = {"bootstrap": bootstrap,"max_features":max_features}
custom_cv = ShuffleSplit(n_splits=10, test_size=0.2)

# run randomized search
grid_search = GridSearchCV(etc, param_grid=param_grid ,cv=custom_cv)
start = time()
grid_search.fit(X, y)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(grid_search.cv_results_['params'])))
print("")
score.report_single(grid_search.cv_results_)

GridSearchCV duro 33.43 segundos para 8 candidatos a hyper-parametros.

Model with rank: 1
Mean training score: 0.705 (std: 0.005)
Mean validation score: 0.469 (std: 0.019)
Mean score time: 0.122
Parameters: {'max_features': None, 'bootstrap': True}

Model with rank: 2
Mean training score: 0.707 (std: 0.004)
Mean validation score: 0.466 (std: 0.018)
Mean score time: 0.122
Parameters: {'max_features': 'sqrt', 'bootstrap': True}

Model with rank: 3
Mean training score: 0.707 (std: 0.005)
Mean validation score: 0.465 (std: 0.015)
Mean score time: 0.122
Parameters: {'max_features': 'log2', 'bootstrap': True}

Model with rank: 4
Mean training score: 0.706 (std: 0.005)
Mean validation score: 0.465 (std: 0.019)
Mean score time: 0.122
Parameters: {'max_features': 'auto', 'bootstrap': True}

Model with rank: 5
Mean training score: 0.714 (std: 0.005)
Mean validation score: 0.464 (std: 0.017)
Mean score time: 0.122
Parameters: {'max_features': 'log2', 'bootstrap': False}

Model with rank: 6
Mean 

<IPython.core.display.Javascript object>